In [ ]:
import numpy as np
import pandas as pd

def cvr(

        stock_price_0: float,
        rate_risk_free: float,
        volatility: float,
        duration: float,
        threshold_price: float,
        payout: float,
        simulations: int,
        trading_days: int = 252,
        consecutive_days: int = 30,
) -> float:

    n = int(duration * trading_days)
    dt = 1 / trading_days

    brownian_drift = ( rate_risk_free - 0.5 * volatility**2 ) * dt

    volatility_jump = volatility * np.sqrt(dt)

    current_total = 0
    trigger_count = 0
    df_results = []

    # loop thru simulations
    for i in range( simulations ):

        z = np.random.standard_normal(n)

        daily_return = brownian_drift + volatility_jump * z

        cum_return= np.cumprod( np.concatenate(([1.0], np.exp(daily_return))) )

        path = stock_price_0 * cum_return[1:]

        trigger_found = False

        # checking for trigger
        for j in range( consecutive_days - 1, n ):

            avg_price = np.mean(path[j - consecutive_days + 1 : j + 1])
            # print(avg_price)

            if avg_price >= threshold_price:
                trigger_found = True
                trigger_step = j + 1
                time_to_trigger = trigger_step * dt
                trigger_count += 1
                break

        payoff = 0
        if trigger_found:
            payoff = payout * np.exp( - rate_risk_free * time_to_trigger)
        current_total += payoff

        df_results.append( {
            'path': i + 1,
            'trigger':  'Yes' if trigger_found else 'No',
            'time to trigger': time_to_trigger if trigger_found else np.nan,
            'payout':  payoff
        })

    cvr = current_total / simulations

    return cvr
    # pd.DataFrame(df_results)

stock_price_0 = 95.00
rate_risk_free = 0.04
volatility = 0.35
duration = 2.0
threshold_price = 120.00
payout = 18.00
simulations = 100000

cvr_price  = cvr(
    stock_price_0,
    rate_risk_free,
    volatility,
    duration,
    threshold_price,
    payout,
    simulations
)

print(f'Current GEN price: ', stock_price_0)
print(f'Trigger price: ', threshold_price)
print(f'CVR: ', cvr_price)


Current GEN price:  95.0
Trigger price:  120.0
CVR:  8.724673902767702
